In [292]:
import numpy as np
from math import pi, gcd
import sympy as sp
from sympy.physics.quantum import TensorProduct
import pandas as pd
import random
try:
    from qiskit import *
    from matplotlib.pyplot import plot,show
except:
    ! python -m pip install matplotlib pylatexenc
    ! python -m pip install qiskit
from qiskit.circuit import *
from qiskit.extensions import *
from qiskit.circuit.library import *
from qiskit.extensions.simulator.snapshot import snapshot
from qiskit.quantum_info.operators import Operator
from qiskit.extensions.simulator.snapshot import snapshot
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller
from scipy import optimize
from matplotlib.pyplot import plot,show
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

In [250]:
from scipy.stats import unitary_group
x = unitary_group.rvs(4)
sp.Matrix(x)

Matrix([
[  0.0788682101922253 + 0.164058453170559*I,  -0.0158224429068613 - 0.0334176364610215*I,   0.588679991509699 + 0.163639754637377*I, -0.761961118418323 + 0.107660083388574*I],
[-0.00584717134384023 + 0.359683549888167*I,      -0.4116580641717 - 0.708329253042642*I,  -0.278466830070751 - 0.285227668991334*I, -0.174185975815889 - 0.100804311588177*I],
[ -0.0652727233934901 + 0.693689817230296*I,     0.408992449534322 + 0.246917853652611*I, -0.426064640413608 + 0.0825657428671443*I, -0.149295772391245 + 0.275051361500757*I],
[   0.468186573368146 + 0.364408322436617*I, -0.000561518648755852 + 0.314955002863249*I,   0.242855516041493 - 0.469523501342284*I,  0.128874070955251 - 0.502763653992476*I]])

In [275]:
x11, x12, x13, x14 = sp.symbols('x11, x12, x13, x14')
x21, x22, x23, x24 = sp.symbols('x21, x22, x23, x24')
x31, x32, x33, x34 = sp.symbols('x31, x32, x33, x34')
x41, x42, x43, x44 = sp.symbols('x41, x42, x43, x44')

K = sp.Matrix([[x11, x12, x13, x14],
              [x21, x22, x23, x24],
              [x31, x32, x33, x34],
              [x41, x42, x43, x44]])

K = (1/2)*sp.Matrix([[2j, 0, 0, 0],
              [0, 2j, 0, 0],
              [0, 0, 2j, 0],
              [0, 0, 0, 2j]])

a1, b1, a2, b2, a3, b3 = sp.symbols('a1, b1, a2, b2, a3, b3')
x1 = sp.Matrix([[a1],[b1]])
x2 = sp.Matrix([[a2],[b2]])
x3 = sp.Matrix([[a3],[b3]])


In [285]:
def case1(K, x1, x2, x3):
    X12 = TensorProduct(x1, x2)
    X = TensorProduct(X12, x3)

    O1 = TensorProduct(sp.eye(2), K)
    state1 = O1.dot(X)

    O2 = TensorProduct(K, sp.eye(2))
    state2 = O2.dot(state1)
    res1 = sp.Matrix(state2)
    return res1

In [287]:
def case2(K, x1, x2, x3):
    X12 = TensorProduct(x1, x2)
    X = TensorProduct(X12, x3)

    O1 = TensorProduct(K, sp.eye(2))
    state1 = O1.dot(X)

    O2 = TensorProduct(sp.eye(2), K)
    state2 = O2.dot(state1)
    res2 = sp.Matrix(state2)
    return res2

In [289]:
def cases_equal(res1, res2):
    return res1 == res2

In [454]:
def is_unitary(m):
    return np.allclose(np.eye(m.shape[0]), m.H * m)

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def get_unitary():
    matrix = []
    Matrix = np.matrix([[1],[1]])
    while not (is_unitary(Matrix) and check_symmetric(Matrix)):
        
        matrix = []
        for i in range(4):
            row = []
            for j in range( 4):
                row.append(0)
            matrix.append(row)
            
        index = 0
        for i in range(4):
            for j in range(index, 4):
                choice = [0, (1/np.sqrt(2)), 0.5, 1, np.sqrt(2)]
                value = choice[random.randint(0, 4)]
                matrix[i][j] = value
                if i != j:
                    matrix[j][i] = value
            index = index + 1
    return matrix

In [448]:
matrix = []
for i in range(4):
    row = []
    for j in range( 4):
        row.append(0)
    matrix.append(row)

index = 0
for i in range(4):
    for j in range(index, 4):
        choice = [0, (1/np.sqrt(2)), 0.5, 1, np.sqrt(2)]
        value = choice[random.randint(0, 4)]
        matrix[i][j] = value
        if i != j:
            matrix[j][i] = value
    index = index + 1
    

sp.Matrix(matrix)

Matrix([
[  0,               0.5,               0.5,                 1],
[0.5,                 1, 0.707106781186547,   1.4142135623731],
[0.5, 0.707106781186547,   1.4142135623731, 0.707106781186547],
[  1,   1.4142135623731, 0.707106781186547, 0.707106781186547]])

In [455]:
m = get_unitary()
K = sp.Matrix(m)
sp.Matrix(m)

KeyboardInterrupt: 

In [ ]:
K = sp.Matrix([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 0, 1],
              [0, 0, 1, 0]])

while not cases_equal(case1(K, x1, x2, x3), case2(K, x1, x2, x3)):
    m = get_unitary()
    K = sp.Matrix(m)
sp.Matrix(m)

In [393]:
K = sp.Matrix([[0, 0, 1, 0],
              [0, 0, 0, 1],
              [1, 0, 0, 0],
              [0, 1, 0, 0]])
cases_equal(case1(K, x1, x2, x3), case2(K, x1, x2, x3))

True

In [394]:
case1(K, x1, x2, x3)

Matrix([
[a3*b1*b2],
[b1*b2*b3],
[a2*a3*b1],
[a2*b1*b3],
[a1*a3*b2],
[a1*b2*b3],
[a1*a2*a3],
[a1*a2*b3]])

In [395]:
case2(K, x1, x2, x3)

Matrix([
[a3*b1*b2],
[b1*b2*b3],
[a2*a3*b1],
[a2*b1*b3],
[a1*a3*b2],
[a1*b2*b3],
[a1*a2*a3],
[a1*a2*b3]])

In [384]:
res1[0]

-1.0*a1*a2*a3

In [385]:
res2[0]

-1.0*a1*a2*a3

In [270]:
res1[1]

x11*(a1*a2*a3*x21 + a1*a2*b3*x22 + a1*a3*b2*x23 + a1*b2*b3*x24) + x12*(a1*a2*a3*x41 + a1*a2*b3*x42 + a1*a3*b2*x43 + a1*b2*b3*x44) + x13*(a2*a3*b1*x21 + a2*b1*b3*x22 + a3*b1*b2*x23 + b1*b2*b3*x24) + x14*(a2*a3*b1*x41 + a2*b1*b3*x42 + a3*b1*b2*x43 + b1*b2*b3*x44)

In [271]:
res2[1]

x21*(a1*a2*a3*x11 + a1*a3*b2*x12 + a2*a3*b1*x13 + a3*b1*b2*x14) + x22*(a1*a2*b3*x11 + a1*b2*b3*x12 + a2*b1*b3*x13 + b1*b2*b3*x14) + x23*(a1*a2*a3*x21 + a1*a3*b2*x22 + a2*a3*b1*x23 + a3*b1*b2*x24) + x24*(a1*a2*b3*x21 + a1*b2*b3*x22 + a2*b1*b3*x23 + b1*b2*b3*x24)

In [272]:
res1[2]

x21*(a1*a2*a3*x11 + a1*a2*b3*x12 + a1*a3*b2*x13 + a1*b2*b3*x14) + x22*(a1*a2*a3*x31 + a1*a2*b3*x32 + a1*a3*b2*x33 + a1*b2*b3*x34) + x23*(a2*a3*b1*x11 + a2*b1*b3*x12 + a3*b1*b2*x13 + b1*b2*b3*x14) + x24*(a2*a3*b1*x31 + a2*b1*b3*x32 + a3*b1*b2*x33 + b1*b2*b3*x34)

In [273]:
res2[2]

x31*(a1*a2*a3*x11 + a1*a3*b2*x12 + a2*a3*b1*x13 + a3*b1*b2*x14) + x32*(a1*a2*b3*x11 + a1*b2*b3*x12 + a2*b1*b3*x13 + b1*b2*b3*x14) + x33*(a1*a2*a3*x21 + a1*a3*b2*x22 + a2*a3*b1*x23 + a3*b1*b2*x24) + x34*(a1*a2*b3*x21 + a1*b2*b3*x22 + a2*b1*b3*x23 + b1*b2*b3*x24)

In [396]:
# matrix = []
# for i in range(8):
#     row = []
#     for j in range(8):
#         if i != j:
#             row.append(0)
#         else:
#             row.append(np.exp(pi*2j*(1/4)))
#     matrix.append(row)

matrix = [[0, 0, 1, 0],
              [0, 0, 0, 1],
              [1, 0, 0, 0],
              [0, 1, 0, 0]]
       
K = UnitaryGate(Operator(matrix), label="K")
K 


# K = UnitaryGate(
#     Operator([[1,0,0,0,0,0,0,1],
#               [0,1,0,0,0,0,0,0],
#               [0,0,1,0,0,0,0,0],
#               [0,0,0,1,0,0,0,0],
#               [0,0,0,0,1,0,0,0],
#               [0,0,0,0,0,1,0,0],
#               [0,0,0,0,0,0,1,0],
#               [1,0,0,0,0,0,0,1]]), label="K")

Instruction(name='unitary', num_qubits=2, num_clbits=0, params=[array([[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]])])

In [397]:
n = 2
q = QuantumRegister(2*n-1)  
c = ClassicalRegister(2*n-1) 
qc = QuantumCircuit(q,c) 

qc.x(q[0])
qc.x(q[1])

qc.append(K, [q[0], q[1]])
qc.append(K, [q[1], q[2]])

qc.barrier()
qc.measure(q, c)
qc.draw()


┌───┐┌────┐       ░ ┌─┐      
q246_0: ┤ X ├┤0   ├───────░─┤M├──────
        ├───┤│  K │┌────┐ ░ └╥┘┌─┐   
q246_1: ┤ X ├┤1   ├┤0   ├─░──╫─┤M├───
        └───┘└────┘│  K │ ░  ║ └╥┘┌─┐
q246_2: ───────────┤1   ├─░──╫──╫─┤M├
                   └────┘ ░  ║  ║ └╥┘
 c75_0: ═════════════════════╩══╬══╬═
                                ║  ║ 
 c75_1: ════════════════════════╩══╬═
                                   ║ 
 c75_2: ═══════════════════════════╩═

In [398]:
backend = BasicAer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
res = dict(job.result().get_counts(qc))
res

{'101': 1024}

In [399]:
n = 2
q = QuantumRegister(2*n-1)  
c = ClassicalRegister(2*n-1) 
qc = QuantumCircuit(q,c) 

qc.x(q[0])
qc.x(q[1])

qc.append(K, [q[1], q[2]])
qc.append(K, [q[0], q[1]])

qc.barrier()
qc.measure(q, c)
qc.draw()

┌───┐      ┌────┐ ░ ┌─┐      
q251_0: ┤ X ├──────┤0   ├─░─┤M├──────
        ├───┤┌────┐│  K │ ░ └╥┘┌─┐   
q251_1: ┤ X ├┤0   ├┤1   ├─░──╫─┤M├───
        └───┘│  K │└────┘ ░  ║ └╥┘┌─┐
q251_2: ─────┤1   ├───────░──╫──╫─┤M├
             └────┘       ░  ║  ║ └╥┘
 c76_0: ═════════════════════╩══╬══╬═
                                ║  ║ 
 c76_1: ════════════════════════╩══╬═
                                   ║ 
 c76_2: ═══════════════════════════╩═

In [400]:
backend = BasicAer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
res = dict(job.result().get_counts(qc))
res

{'101': 1024}

In [204]:
matrix = []
for i in range(4):
    row = []
    for j in range(4):
        if i != j:
            row.append(0)
        else:
            row.append(np.exp(pi*2j*(1/4)))
    matrix.append(row)

matrix
        
K = UnitaryGate(Operator(matrix), label="K")
K 

Instruction(name='unitary', num_qubits=2, num_clbits=0, params=[array([[6.123234e-17+1.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 6.123234e-17+1.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 6.123234e-17+1.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        6.123234e-17+1.j]])])

In [203]:
x = np.exp(pi*2j*(1/4))
print(np.csingle(x))

(6.123234e-17+1j)


In [210]:
n = 3
q = QuantumRegister(3)  
c = ClassicalRegister(3) 
qc = QuantumCircuit(q,c) 

qc.x(q[0])

qc.append(K, [q[0], q[1]])
qc.append(K, [q[0], q[1]])

qc.barrier()
qc.measure(q, c)
qc.draw()


┌───┐ ░ ┌─┐      
q220_0: ┤ X ├─░─┤M├──────
        └───┘ ░ └╥┘┌─┐   
q220_1: ──────░──╫─┤M├───
              ░  ║ └╥┘┌─┐
q220_2: ──────░──╫──╫─┤M├
              ░  ║  ║ └╥┘
 c68_0: ═════════╩══╬══╬═
                    ║  ║ 
 c68_1: ════════════╩══╬═
                       ║ 
 c68_2: ═══════════════╩═

In [211]:
backend = BasicAer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
res = dict(job.result().get_counts(qc))
res

{'001': 1024}